# Zero-Shot Anomaly Detection with DINOv3


In [1]:
# Configuration
import json
import random
import torch
import numpy as np
from pathlib import Path
from sklearn.metrics import roc_auc_score
from transformers import AutoImageProcessor, AutoModel
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve()
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from data.data_loader import DatasetLoader
from data.transform import ProcessorTransform
from evaluation.distance import calculate_distance

seed = 42
random.seed(seed)


# Experiment configuration
config = {
    "seed": 42,
    "title": "zero shot anomaly detection",
    "output_path": "outputs/",
    "object": [
        "bottle",
        "cable",
        "capsule",
        "carpet",
        "grid",
        "hazelnut",
        "leather",
        "metal_nut",
        "pill",
        "screw",
        "tile",
        "toothbrush",
        "transistor",
        "wood",
        "zipper"
  ],  # Start with one object for testing
    "experiments": [
        {
            "ref_img_count": 50,
            "image_size": 224,
            "vit_model": "dinov3_vits16",
            "distance": "mahalanobis",
            "top_n": 5,
            "ref_aggregation_method": ""
        }
    ]
}

experiment_param = config["experiments"][0]
print(f"Experiment config: {experiment_param}")


Experiment config: {'ref_img_count': 50, 'image_size': 224, 'vit_model': 'dinov3_vits16', 'distance': 'mahalanobis', 'top_n': 5, 'ref_aggregation_method': ''}


In [2]:
from main import compute_scores_dinov3

print("Loading DINOv3 model...")
processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m")
model = AutoModel.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m")
model.eval()

PROJECT_ROOT = Path().resolve()

for object in config["object"]:
    # Setup paths
    test_path = PROJECT_ROOT / 'data' / 'mvtec_anomaly_detection' / 'bottle' / 'test'
    ref_path = PROJECT_ROOT / 'data' / 'mvtec_anomaly_detection' / 'bottle' / 'train'

    # Setup datasets
    ref_dataset = DatasetLoader(ref_path, transform=ProcessorTransform(processor))
    test_dataset = DatasetLoader(test_path, transform=ProcessorTransform(processor))

    score, labels = compute_scores_dinov3(ref_path, test_path, experiment_param, model, processor)

    auc = roc_auc_score(labels, score)

    print(f"{object} AUC: {auc}")


Loading DINOv3 model...


Loading weights:   0%|          | 0/211 [00:00<?, ?it/s]

bottle AUC: 1.0
cable AUC: 1.0
capsule AUC: 1.0
carpet AUC: 1.0
grid AUC: 1.0
hazelnut AUC: 1.0
leather AUC: 1.0
metal_nut AUC: 1.0
pill AUC: 1.0
screw AUC: 1.0
tile AUC: 1.0
toothbrush AUC: 1.0
transistor AUC: 1.0
wood AUC: 1.0
zipper AUC: 1.0
